In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.base import clone
import joblib
import pickle
import matplotlib.pyplot as plt
import doubleml as dml
import patsy
import plotly.express as px
from plotly.subplots import make_subplots
from plotly.offline import plot

from statsmodels.tsa.seasonal import seasonal_decompose

In [2]:
dates= pd.date_range('2017-01-01','2021-07-01' , freq='1M')-pd.offsets.MonthBegin(1)
dates=[date_obj.strftime('%Y-%m-%d') for date_obj in dates]

var_quel=["emploi_quelconque_t_plus_3","emploi_quelconque_t_plus_6",  "emploi_quelconque_t_plus_12" ,"emploi_quelconque_t_plus_18" ,"emploi_quelconque_t_plus_24","emploi_quelconque_t_plus_36"]

x_cols=["age", "anciennete","sexe","zone_urbaine","handicap" ]

path="C:/Users/dalil.youcefi/Documents/Formation et retour à l'emploi/Données"

In [3]:
li_poids=[]
for cohorte in dates:
    df=pd.read_parquet(path+"/Prepro/Toutes Form/prepro_data_"+cohorte+".parquet",engine='pyarrow')
    li_poids.append(df.shape[0])

In [4]:


dic_irm={}





dic_irm["cohorte"]=dates



for var in var_quel:
    irm_coef=[]

    irm_upp=[]
    irm_low=[]
  
    irm_std=[]
    
    
    for cohorte in dates:
     
      
        
        dml_irm_obj=joblib.load( path+"/Résultats/Python/IRM/ATT/dml_irm_att_"+cohorte+"_"+var+".joblib")
        irm_coef.append(dml_irm_obj.summary.coef)
        irm_low.append(dml_irm_obj.confint()["2.5 %"])
        irm_upp.append(dml_irm_obj.confint()["97.5 %"])
        irm_std.append(dml_irm_obj.summary["std err"])
        
        
   
    dic_irm["coef_"+var]=irm_coef
    dic_irm["upp_"+var]=irm_upp
    dic_irm["low_"+var]=irm_low
    dic_irm["std_"+var]=irm_std
    
    
 

   
df_irm=pd.DataFrame.from_dict(dic_irm,orient="columns")





In [21]:
dml_irm_obj.summary.coef

entree_formation    0.099528
Name: coef, dtype: float64

In [29]:
df_irm["poids"]=li_poids
dic_avg={}



irm_coef=[]

irm_upp=[]
irm_low=[]

irm_std=[]



def weighted_average(dataframe, value, weight):
    val = dataframe[value]
    wt = dataframe[weight]
    return (val * wt).sum() / wt.sum()
 


dic_avg={}


dic_avg["temps"]=["6","12","18","24"]




var="emploi_quelconque_t_plus_6"
    

period=pd.date_range('2017-01-01','2019-10-01' , freq='1M')-pd.offsets.MonthBegin(1)
period=[date_obj.strftime('%Y-%m-%d') for date_obj in period]
    
irm_coef.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"coef_"+var,"poids"))
irm_upp.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"upp_"+var,"poids"))
irm_low.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"low_"+var,"poids"))
irm_std.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"std_"+var,"poids"))


var="emploi_quelconque_t_plus_12"
    

period=pd.date_range('2017-01-01','2019-10-01' , freq='1M')-pd.offsets.MonthBegin(1)
period=[date_obj.strftime('%Y-%m-%d') for date_obj in period] 
irm_coef.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"coef_"+var,"poids"))
irm_upp.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"upp_"+var,"poids"))
irm_low.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"low_"+var,"poids"))
irm_std.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"std_"+var,"poids"))
var="emploi_quelconque_t_plus_18"
    

period=pd.date_range('2017-01-01','2019-04-01' , freq='1M')-pd.offsets.MonthBegin(1)
period=[date_obj.strftime('%Y-%m-%d') for date_obj in period]   
irm_coef.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"coef_"+var,"poids"))
irm_upp.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"upp_"+var,"poids"))
irm_low.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"low_"+var,"poids"))
irm_std.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"std_"+var,"poids"))
var="emploi_quelconque_t_plus_24"
    

period=pd.date_range('2017-01-01','2019-10-01' , freq='1M')-pd.offsets.MonthBegin(1)
period=[date_obj.strftime('%Y-%m-%d') for date_obj in period]    
irm_coef.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"coef_"+var,"poids"))
irm_upp.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"upp_"+var,"poids"))
irm_low.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"low_"+var,"poids"))
irm_std.append(weighted_average(df_irm[df_irm.cohorte.isin(period)],"std_"+var,"poids"))
    
    


dic_avg["irm_coef"]=irm_coef
dic_avg["irm_upp"]=irm_upp
dic_avg["irm_low"]=irm_low
dic_avg["irm_std"]=irm_std
df_avg=pd.DataFrame.from_dict(dic_avg,orient="columns")

df_avg=df_avg*100

df_avg["temps"]=["6","12","18","24"]    

In [18]:
df_irm[df_irm.cohorte.isin(period)]

,cohorte,coef_emploi_quelconque_t_plus_3,upp_emploi_quelconque_t_plus_3,low_emploi_quelconque_t_plus_3,std_emploi_quelconque_t_plus_3,coef_emploi_quelconque_t_plus_6,upp_emploi_quelconque_t_plus_6,low_emploi_quelconque_t_plus_6,std_emploi_quelconque_t_plus_6,coef_emploi_quelconque_t_plus_12,...,std_emploi_quelconque_t_plus_18,coef_emploi_quelconque_t_plus_24,upp_emploi_quelconque_t_plus_24,low_emploi_quelconque_t_plus_24,std_emploi_quelconque_t_plus_24,coef_emploi_quelconque_t_plus_36,upp_emploi_quelconque_t_plus_36,low_emploi_quelconque_t_plus_36,std_emploi_quelconque_t_plus_36,poids
0,2017-01-01,"entree_formation -0.018755 Name: coef, dtype...","entree_formation -0.011425 Name: 97.5 %, dty...","entree_formation -0.026086 Name: 2.5 %, dtyp...","entree_formation 0.00374 Name: std err, dty...","entree_formation 0.028127 Name: coef, dtype...","entree_formation 0.037224 Name: 97.5 %, dty...","entree_formation 0.019029 Name: 2.5 %, dtyp...","entree_formation 0.004642 Name: std err, dt...","entree_formation 0.08481 Name: coef, dtype:...",...,"entree_formation 0.005169 Name: std err, dt...","entree_formation 0.123037 Name: coef, dtype...","entree_formation 0.133024 Name: 97.5 %, dty...","entree_formation 0.113049 Name: 2.5 %, dtyp...","entree_formation 0.005096 Name: std err, dt...","entree_formation 0.125838 Name: coef, dtype...","entree_formation 0.135466 Name: 97.5 %, dty...","entree_formation 0.11621 Name: 2.5 %, dtype...","entree_formation 0.004912 Name: std err, dt...",55671
1,2017-02-01,"entree_formation 0.001502 Name: coef, dtype...","entree_formation 0.006814 Name: 97.5 %, dty...","entree_formation -0.003809 Name: 2.5 %, dtyp...","entree_formation 0.00271 Name: std err, dty...","entree_formation 0.079392 Name: coef, dtype...","entree_formation 0.086042 Name: 97.5 %, dty...","entree_formation 0.072743 Name: 2.5 %, dtyp...","entree_formation 0.003393 Name: std err, dt...","entree_formation 0.119467 Name: coef, dtype...",...,"entree_formation 0.003608 Name: std err, dt...","entree_formation 0.143094 Name: coef, dtype...","entree_formation 0.150012 Name: 97.5 %, dty...","entree_formation 0.136175 Name: 2.5 %, dtyp...","entree_formation 0.00353 Name: std err, dty...","entree_formation 0.140821 Name: coef, dtype...","entree_formation 0.147422 Name: 97.5 %, dty...","entree_formation 0.134221 Name: 2.5 %, dtyp...","entree_formation 0.003368 Name: std err, dt...",69294
2,2017-03-01,"entree_formation 0.006138 Name: coef, dtype...","entree_formation 0.011441 Name: 97.5 %, dty...","entree_formation 0.000835 Name: 2.5 %, dtyp...","entree_formation 0.002706 Name: std err, dt...","entree_formation 0.077655 Name: coef, dtype...","entree_formation 0.084097 Name: 97.5 %, dty...","entree_formation 0.071213 Name: 2.5 %, dtyp...","entree_formation 0.003287 Name: std err, dt...","entree_formation 0.129049 Name: coef, dtype...",...,"entree_formation 0.003514 Name: std err, dt...","entree_formation 0.146663 Name: coef, dtype...","entree_formation 0.153388 Name: 97.5 %, dty...","entree_formation 0.139939 Name: 2.5 %, dtyp...","entree_formation 0.003431 Name: std err, dt...","entree_formation 0.140553 Name: coef, dtype...","entree_formation 0.146995 Name: 97.5 %, dty...","entree_formation 0.134111 Name: 2.5 %, dtyp...","entree_formation 0.003287 Name: std err, dt...",72574
3,2017-04-01,"entree_formation 0.018629 Name: coef, dtype...","entree_formation 0.024415 Name: 97.5 %, dty...","entree_formation 0.012844 Name: 2.5 %, dtyp...","entree_formation 0.002952 Name: std err, dt...","entree_formation 0.082446 Name: coef, dtype...","entree_formation 0.089479 Name: 97.5 %, dty...","entree_formation 0.075413 Name: 2.5 %, dtyp...","entree_formation 0.003588 Name: std err, dt...","entree_formation 0.127531 Name: coef, dtype...",...,"entree_formation 0.003796 Name: std err, dt...","entree_formation 0.151302 Name: coef, dtype...","entree_formation 0.15858 Name: 97.5 %, dtyp...","entree_formation 0.144024 Name: 2.5 %, dtyp...","entree_formation 0.003714 Name: st

In [30]:
df_avg=df_avg.astype(float)

In [31]:
df_avg["match_coef"]=[0.6,5.7,8.8,9.2]
df_avg["match_std"]=[0.27,0.29,0.29,0.29]

In [32]:
np.round(df_avg,2)

,temps,irm_coef,irm_upp,irm_low,irm_std,match_coef,match_std
0,6.0,3.16,3.81,2.51,0.33,0.6,0.27
1,12.0,8.48,9.18,7.77,0.36,5.7,0.29
2,18.0,11.81,12.52,11.09,0.37,8.8,0.29
3,24.0,11.93,12.63,11.23,0.36,9.2,0.29
